<a href="https://colab.research.google.com/github/rishadb/sample_codes/blob/main/1NN0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
import matplotlib.pyplot as plt
#torch, sklearn - logistic ression

In [ ]:
####         TORCH             ####
device = "cuda" if torch.cuda.is_available() else "cpu"
tens1 = torch.tensor([[1,2,3],[4,5,6],[7,8,9]], dtype = torch.float32, device = device, requires_grad= True)
print(tens1.device, tens1.requires_grad, tens1.dtype, tens1.shape)

#initialization methods:
x1 = torch.empty(size =(3,3)) # can contain non initioalized vals
x2 = torch.zeros(3,2)
x3 = torch.rand((3,2))
x4 = torch.eye(5,5) # identity matrix
x5 = torch.arange(start = 1, end = 5, step = 1) # gives [1,2,3,4]
x5 = torch.arange(4) # start =0, step =1 default
x6 = torch.linspace(start = 0.1, end = 1, steps = 6) 
x6.normal_(mean = 0, std = 1)
x6.uniform_(1,3) # make values inside 1 and 3
print(x1, x2, x3, x4, x5, x6)

#conversions
print(x5.bool(), x5.short(), x5.long(), x5.float(), x5.double())
np_arr = x5.numpy()
x5 = torch.from_numpy(np_arr)

#dimensional ops:
torch.sum(x4, dim = 0) # adds all elements in row 
vals, indices = torch.max(x4, dim = 0) # gives tensor of max vals


##TORCH OPS
a = torch.ones(4,4)
b = torch.ones(4,4)
print(type(a))
print(a+b, a-b, a*b, a**b, a.add_(b), a.mm(b), a<b)
print(torch.dot(a[1], b[2])) # torch.dot only support 1d op

#Batch multiplication (batch, a, b) bmm (batch, a, c)
at = torch.rand(1,4,3)
bt = torch.rand(1,3,2)
torch.bmm(at,bt)

#broadcasting
x=torch.empty(5,1,4,1)
y=torch.empty(  5,1,1)
print((x+y).size())
x=torch.empty(5,1,4,1)
y=torch.empty(  3,1,1)
print((x+y).size())
x ** y

#sorting
a = torch.rand(4,3)
sorted, sort_indices = torch.sort(a, dim = 0, descending = False)
print(a, sorted, sort_indices)

#clamping
print(torch.clamp(x, min = 0, max = 3)) # make all elements less than 0 to 0 and greater than 3 to 3

#any, all method
x = torch.tensor([1,0,3,5,0], dtype = torch.bool)
x.any(), torch.any(x), torch.all(x) #check if any of x is true and all is true

#indexing
x = torch.rand(2,3,4)
x[1,2,3] 
x, x[[1,[1,2]]] #fancy indexing
  #advance indexing
x[(x > 1) | (x < 0.5)] #gives tensor of all elements woth condition
x[x.remainder(2) == 0]

# other ops:
x, torch.where(x > .5, x, x **2 ) #replace element wrt condition
torch.tensor([0,0,2,3,2,1]).unique() #returns unique elements tensor
x.ndimension(), x.numel(), a.t() # dimensions and number of elements, transpose of x
print(x.abs()) # abs val
print(torch.argmax(x, dim = 0), x.max(dim =0)) # retrurns index of max only
print(torch.mean(x.float(), dim = 0)) #mean along axis
a = torch.rand(2,3,4)
b= torch.rand(2,3,4)
print(torch.eq(a, b), a==b) #returns bool matrix 

#Reshaping tensor:
x.reshape(24) #if not contiguous, makes a copy and moveon, slower
x.view(2,3,2,2) # x needs to be contiguous in memory, but this is faster
x_t = torch.transpose(x, 0, 1) #transpose is non-contig
  #to make non-contig, contiguous, 
x_t.contiguous().view(24)
x.view(-1,6,2) # gives a tensor with 12 in one dimension and other figured out automatically
a = a.view(-1,2)
  # store tensot in GPU for: 1. faster, 2. to store nparray and tensors in defferent address
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
x = torch.ones(5,3, device = device)
x = x.to("cpu") # note: np cant convert tensor back to numpy in GPU, so
x = x.numpy() 


#concatinate matrices
a = torch.rand(3,6)
b = torch.rand(3,4)
torch.cat((a,b), dim = 1)

#flatten the tensor
x = torch.rand(1,2,3,4)
x_flat = x.view(1,-1)
x_1D = x.view(-1)

#switch axis
x.permute(0,2,1,3) #by changing the indeces of shape
x_1D.unsqueeze(0), x_1D.unsqueeze(1) #expand 1D to 1,D and D,1
x_11D = x_1D.unsqueeze(0).unsqueeze(1) # gives 1,1,D 
x_1D = x_11D.unsqueeze(0)

#gradient, Autorgad package
x = torch.randn(3,2, requires_grad = True)
y = x+2
z= x*2+3
z = z.mean()
w = x **2 + 1
e = w.mean()
z.backward()
print(x.grad)
"""to skip the tensor from grad tracking:
 x.requires_grad_(False) OR x.detach() OR do ops as: with torch.no_grad():"""
 # calling backwards again will add up in x.grad. so clear grad by x.zero_grad()

#best practices#
# avoid using mutable object as default value for optional params with below
def muta(lst = None):
  if lst ==None:
    lst = []
  lst.append(1)
  lst.append(2)
  return lst
muta()
muta()
#time complexity for datastructures ...
#list(when order and frequency of elements is important), set(only presence of element is imp, ), dict
#use comprehensions


In [ ]:
#SKLearn - Logistic regression

"""
0 - prepare data, imports, device
1 - HP, Data-dataset, dataloader
2 - create NNarch and initialize model
3 - construct loss func, optimiser
4 - Train Loop
  1- forward pass: prediction and loss calc
  2- backward pass: gradients
  3- Update wights by optimizer, zero_grad
5 - Check accuracy of train and test with no grad tracking

"""

##LINEAR REGRESSION - SK learn
from sklearn import datasets # to make train datasets
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler #to scale dataset features
from sklearn.model_selection import train_test_split # to make train and test set
import torch
import numpy as np



#0- make datasets
bc = datasets.load_breast_cancer()
X, Y = bc.data, bc.target
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 2)
#X_numpy, Y_numpy = datasets.make_regression(n_samples = 100, n_features=1, noise =20, random_state=1)
#X = torch.from_numpy(X_numpy.astype(np.float32))
#Y = torch.from_numpy(Y_numpy.astype(np.float32))

  # Normalizing input
sc = StandardScaler() #normalizing function declared
X_train = sc.fit_transform(X_train) # apply normalization on dataset
X_test = sc.transform(X_test)
n_samples, n_features = X.shape

  #converting np to torch
X_train = torch.from_numpy(X_train.astype(np.float32))
Y_train = torch.from_numpy(Y_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))
Y_test = torch.from_numpy(Y_test.astype(np.float32))

  #fixing y
Y_train = Y_train.view(Y_train.shape[0], 1)
Y_test = Y_test.view(Y_test.shape[0], 1)

#define NNarch:
class LogRegression(nn.Module):
  def __init__(self, n_input_features):
    super().__init__()
    self.linear = nn.Linear(n_input_features, 1)
  def forward(self, x):
    x = torch.sigmoid(self.linear(x))
    return x
model = LogRegression(n_features)

#HP
learning_rate = .01
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)
n_epochs = 10


#training
for epoch in range(n_epochs):
  y_pred = model(X_train)
  loss = criterion(y_pred, Y_train)

  loss.backward()

  optimizer.step()

  optimizer.zero_grad()

#accuracy
with torch.no_grad():
  y_pred = model(X_test)
  y_pred_cls = y_pred.round()
  acc = (Y_test == y_pred_cls).sum()
  accu = acc/Y_test.shape[0]

print(accu)



tensor(0.7632)
